In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 2: Load dataset
df = pd.read_csv("weather.csv")  # Replace with your actual file name
df = df.sort_values('date')      # Optional: ensure data is in chronological order
df.set_index('date', inplace=True)

# Step 3: Select feature and scale it
data = df[['temperature']].values  # Replace 'temperature' with your column name
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Step 4: Prepare sequences for LSTM
def create_sequences(data, window_size=10):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, window_size=10)

# Step 5: Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Step 6: Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Step 7: Train model
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

# Step 8: Make predictions
predicted = model.predict(X_test)
predicted = scaler.inverse_transform(predicted)
actual = scaler.inverse_transform(y_test)

# Step 9: Evaluate
mse = mean_squared_error(actual, predicted)
print("Test MSE:", mse)

# Step 10: Plot
plt.plot(actual, label="Actual")
plt.plot(predicted, label="Predicted")
plt.title("Weather Forecasting")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.legend()
plt.show()